<a href="https://colab.research.google.com/github/siddheshdosi/Deep-Learning-Image_Classfication/blob/master/TransferLearning_MobileNetV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#%tensorflow_version 1.x
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras import Model
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

import cv2

import sys
import numpy as np
import csv
import math

import matplotlib.pyplot as plt

### Import Dataset

In [ ]:
# Class names for different classes
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer','dog', 'frog', 'horse', 'ship', 'truck']

# Load training data, labels; and testing data and their true labels
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
print ('Training data seize:', train_images.shape, 'Test data size', test_images.shape)

# Normalize pixel values between -1 and 1
train_images = train_images / 127.5 - 1 
test_images = test_images / 127.5 - 1 

# one hot encoding 
ytrain = to_categorical(train_labels)
ytest = to_categorical(test_labels)

### Visualize dataset

In [ ]:
%matplotlib inline
#Show first 25 training images below
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i][0]])

### Resize images for use with MobileNetV2

In [ ]:
# Upsize all training and testing images to 96x96 for use with mobile net
minSize = 96 #minimum size requried for mobileNetV2
# You may use cv2 package. Look for function:
#"cv2.resize(<originalImage>, dsize=(minSize, minSize), interpolation=cv2.INTER_AREA)"
# resize train image: You can first initialize a numpy array resized_train_images to store all the resized training images
resized_train_images = np.zeros((50000, minSize, minSize, 3), dtype=np.float32)
# <Write code for resizing>
for i in range(0,50000):
    resized_train_images[i] = cv2.resize(train_images[i], dsize=(minSize, minSize), interpolation=cv2.INTER_AREA)
# resize test image: You can first initialize a numpy array resized_test_images to store all the resized test images
resized_test_images = np.zeros((10000, minSize, minSize, 3), dtype=np.float32)
# <Write code for resizing>
for i in range(0,10000):
    resized_test_images[i] = cv2.resize(test_images[i], dsize=(minSize, minSize), interpolation=cv2.INTER_AREA)

In [ ]:
del train_images
del test_images
del train_labels
del test_labels

In [ ]:
#defining training and test sets
#x_train,x_val,y_train,y_val=train_test_split(resized_train_images,ytrain,test_size=.3)

### Download MobileNetV2 model



In [ ]:
#<Write code for downloading MobileNetV2>
IMG_SHAPE=(minSize,minSize,3)
base_model = MobileNetV2(input_shape=IMG_SHAPE,include_top=False,weights='imagenet')
#base_model = MobileNetV2(include_top=False, weights='imagenet')

### Add custom layers at the end of downloaded model

In [ ]:
#<Write code for adding custom layers>
from keras.layers import Dense,GlobalAveragePooling2D,Flatten,Conv2D,MaxPool2D
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
x=base_model.output
x = Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(3, 3, 1280))(x)
x = MaxPool2D(pool_size=2, strides=2)(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
#x = MaxPool2D(pool_size=2)(x)
#x=GlobalAveragePooling2D()(x)
x=Flatten()(x)
#x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(128,activation='relu')(x) #dense layer 2
x=Dense(64,activation='relu')(x) #dense layer 3
preds=Dense(10,activation='softmax')(x)
model=Model(inputs=base_model.input,outputs=preds)

In [ ]:
model.summary()

### Add loss function, compile and train the model, and check accuracy on test data

In [ ]:
#<Write code>
from keras.metrics import categorical_crossentropy
from keras.layers.core import Dense, Activation
#from keras.optimizers import Adam

from keras.callbacks import Callback

class stopAtLossValue(Callback):

        def on_batch_end(self, batch, logs={}):
            THR = 0.001 #Assign THR with the value at which you want to stop training.
            if logs.get('loss') <= THR:
                 self.model.stop_training = True

callbacks = stopAtLossValue()

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

history = model.fit(resized_train_images,ytrain,batch_size=128,
                   epochs=6,callbacks=[callbacks])

### Extra code for producing different plots

In [ ]:
plt.plot(history.history['accuracy'])
#plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
#plt.legend(['train', 'validation'], loc='upper left')
plt.show()

**1) Write how you extended the MobileNetV2 model (how many layers you added, what type of layers, how many nodes per layer, their activation function etc).**

**2) Plot the loss function value with respect to the epoch number on the training data. How did you decide when to terminate training? How did you decide the mini-batch size for training?**

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
#plt.legend(['train', 'validation'], loc='upper left')
plt.show()

**3) Show accuracy of the trained classifier over the entire testing dataset.**

In [ ]:
from sklearn.metrics import accuracy_score
ypred = model.predict(resized_test_images)

predict = [np.argmax(p) for p in ypred]
actual = [np.argmax(p) for p in ytest] 
print('Test data accuracy : ',accuracy_score(actual,predict))